In [1]:
from scipy import stats
import itertools
import math
import pandas as pd
import numpy as np
from scipy.stats import studentized_range


data = pd.read_csv(r"D:\DataSets\genetherapy.csv")

def ssdev( xs ):
    mu = xs.mean() 
    square_deviation = lambda x : (x - mu) ** 2 
    return sum( map(square_deviation, xs) )


SSW = sum(data.groupby('Therapy').apply(lambda g: ssdev(np.array(g)))) 
# Out: 1302.2666666666669

dfSSW = len(data.expr) - len(data.Therapy.unique()) # Out: 56
MSw = SSW/dfSSW # Out: 186.90555556
SE = math.sqrt(MSw/15) #Out: 1.2451174509525864

combinations = list(itertools.combinations(data.Therapy.unique(), 2))
#Out: [('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]

n_group = len(data.Therapy.unique()) #Out: 4
dfSSW = len(data.expr) - len(data.Therapy.unique()) #Out:56


alpha = 0.05
q_crit = studentized_range.ppf(1-alpha, n_group, dfSSW) # Out: 3.7446781868177195

df = pd.DataFrame(columns=[f'{first} - {second}' for first, second in combinations], 
                 index=['q','meandiff','p-adj','lower','upper'])

means = data.groupby('Therapy').mean().T

for first, second in combinations:
    key = f'{first} - {second}'
    first_second_mean = [means[first].expr, means[second].expr]
    _q = (max(first_second_mean) - min(first_second_mean))/SE
    _meandiff = (means[second].expr - means[first].expr)
    df[key]['q'] = _q
    df[key]['meandiff'] = _meandiff
    df[key]['p-adj'] = 1-studentized_range.cdf(_q, n_group, dfSSW)
    df[key]['upper'] = _meandiff+ q_crit*SE
    df[key]['lower'] = _meandiff- q_crit*SE

df.T

,q,meandiff,p-adj,lower,upper
A - B,0.749595,-0.933333,0.95142,-5.595897,3.729231
A - C,4.283398,-5.333333,0.018886,-9.995897,-0.670769
A - D,5.943214,-7.4,0.000542,-12.062564,-2.737436
B - C,3.533803,-4.4,0.071041,-9.062564,0.262564
B - D,5.19362,-6.466667,0.002944,-11.129231,-1.804103
C - D,1.659817,-2.066667,0.645794,-6.729231,2.595897
